In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import jax
import pandas as pd
import numpy as np
import jax.numpy as jnp
from utilities.adding_noise import adding_noise_to_observations
from jax import random, vmap, jit, grad, lax
import jax
from jax import random
from jax.nn.initializers import normal, zeros
from jax.example_libraries import stax
from jax.example_libraries import optimizers

import pickle
import glob

import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.tri as tri
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm  # Colour map
import matplotlib.animation as animatio
import matplotlib.font_manager
mpl.rcParams['figure.figsize'] = [5, 5/1.3333]
mpl.rcParams['figure.dpi'] = 300
params = {'legend.fontsize': 9, 'axes.labelsize': 9, 'xtick.labelsize':9, 'ytick.labelsize':9}
pylab.rcParams.update(params)
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'

In [2]:
num_test = 500

case = 8
key_noise_train = random.PRNGKey(10 + case)
key_noise_test = random.PRNGKey(20 + case)
key_network = 100
noise_level = 0.02

#? 0.4 Physical model information
n = 32
num_observation = 20  # number of observed points
num_truncated_series = 24
dimension_of_PoI = (n)**2  # external force field

neurons = 5000


## Loading test data


In [3]:

def load_data(name, target_shape=(-1,)):
    return np.reshape(pd.read_csv(name).to_numpy(), target_shape)
train_Observations_synthetic = load_data('data/Train_Observations10000.csv', (10000, -1))
train_Parameters = load_data('data/Train_PoI_samples_d10000.csv', (10000, -1))
train_full_Observations_synthetic = load_data('data/Train_full_Observations10000.csv', (10000, -1))

# train_Observations_synthetic = jnp.repeat(train_Observations_synthetic, repeat_fac, axis=0)
# train_Parameters = jnp.repeat(train_Parameters, repeat_fac, axis=0)

test_Observations_synthetic = load_data('data/Test_Observations' + str(num_test) + '.csv', (num_test, -1))
test_Observations_full = load_data('data/Test_full_Observations' + str(num_test) + '.csv', (num_test, -1))
test_Parameters = load_data('data/Test_PoI_samples_d' + str(num_test) + '.csv', (num_test, -1))

#? 1.1 Add noise (WE DO NOT USE vmap, BE CAREFUL! )
train_Observations = adding_noise_to_observations(train_Observations_synthetic, noise_level, key_noise_train)
test_Observations = adding_noise_to_observations(test_Observations_synthetic, noise_level, key_noise_test)

#? 1.2 Load Eigenvalue, Eigenvectors, observed indices, prematrices
Basis = load_data('data/Basis' + '.csv', (dimension_of_PoI, num_truncated_series))
obs_locations = load_data('data/obs_locations' + '.csv', (num_observation))

Tik_POI = np.loadtxt('data/Tik_solutions.txt', delimiter = ',')

In [4]:
print(Tik_POI.shape, train_full_Observations_synthetic.shape)

(500, 24) (10000, 1024)


## Encoder decoder neural networks

In [5]:
def neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network):
    #? 2.0 Define Obs_2_PoI_NN and PoI_2_Obs_NN architectures
    Obs_2_PoI_NN_init, Obs_2_PoI_NN = stax.serial(
        stax.Dense(neurons, W_init=normal(0.02), b_init=zeros), stax.Relu,
        stax.Dense(num_truncated_series, W_init=normal(0.02), b_init=zeros)
    )

    PoI_2_Obs_NN_init, PoI_2_Obs_NN = stax.serial(
        stax.Dense(neurons, W_init=normal(0.02), b_init=zeros), stax.Relu,
        stax.Dense(32**2, W_init=normal(0.02), b_init=zeros)
    )
    
    #? 2.1 Randomize the weights and bias
    enc_init_rng, dec_init_rng = random.split(random.PRNGKey(key_network))
    
    _, init_Obs_2_PoI_NN_params = Obs_2_PoI_NN_init(enc_init_rng, (num_observation,))
    _, init_PoI_2_Obs_NN_params = PoI_2_Obs_NN_init(dec_init_rng, (num_truncated_series,))

    init_params = init_Obs_2_PoI_NN_params, init_PoI_2_Obs_NN_params
    return Obs_2_PoI_NN, PoI_2_Obs_NN, init_params

def neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network):
    #? 2.0 Define Obs_2_PoI_NN and PoI_2_Obs_NN architectures
    Obs_2_PoI_NN_init, Obs_2_PoI_NN = stax.serial(
        stax.Dense(neurons, W_init=normal(0.02), b_init=zeros), stax.Relu,
        stax.Dense(num_truncated_series, W_init=normal(0.02), b_init=zeros)
    )

    PoI_2_Obs_NN_init, PoI_2_Obs_NN = stax.serial(
        stax.Dense(neurons, W_init=normal(0.02), b_init=zeros), stax.Relu,
        stax.Dense(num_observation, W_init=normal(0.02), b_init=zeros)
    )

    #? 2.1 Randomize the weights and bias
    enc_init_rng, dec_init_rng = random.split(random.PRNGKey(key_network))

    _, init_Obs_2_PoI_NN_params = Obs_2_PoI_NN_init(enc_init_rng, (num_observation,))
    _, init_PoI_2_Obs_NN_params = PoI_2_Obs_NN_init(dec_init_rng, (num_truncated_series,))

    init_params = init_Obs_2_PoI_NN_params, init_PoI_2_Obs_NN_params
    return Obs_2_PoI_NN, PoI_2_Obs_NN, init_params



## Accuracy functions

In [6]:

#? 3.1 Define the main forward solver
def Observation_operator(full_state):
    return full_state[[obs_locations], ].flatten()
    
batched_Observation_operator = jax.jit(vmap(Observation_operator))
    
# ! 4.2 Accuracy rate function
def acc_PoI(preds, true):
    return jnp.mean(jnp.square(Basis @ (preds - true).T)) / jnp.mean(jnp.square(Basis @ true.T)) * 100

def acc_obs(preds, true):
    return jnp.mean(jnp.square(preds - true)) / jnp.mean(jnp.square(true))

batched_acc_PoI = vmap(acc_PoI, in_axes=(0, 0))
batched_acc_obs = vmap(acc_obs, in_axes=(0, 0))

@jit
def accuracy_data_P2_full_O(params, Observations, PoI):
    init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained = params

    Obs_2_PoI_NN_preds = Obs_2_PoI_NN(init_Obs_2_PoI_NN_params_trained, Observations)
    Obs_2_PoI_NN_Err_all = batched_acc_PoI(Obs_2_PoI_NN_preds, PoI)
    
    PoI_2_Obs_NN_preds = batched_Observation_operator(PoI_2_Obs_NN(init_PoI_2_Obs_NN_params_trained, PoI))
    PoI_2_Obs_NN_Err_all = batched_acc_obs(PoI_2_Obs_NN_preds, Observations)
    
    return Obs_2_PoI_NN_Err_all.mean(), PoI_2_Obs_NN_Err_all.mean()

@jit
def accuracy_data_P2_sparse_O(params, Observations, PoI):
    init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained = params

    Obs_2_PoI_NN_preds = Obs_2_PoI_NN(init_Obs_2_PoI_NN_params_trained, Observations)
    Obs_2_PoI_NN_Err_all = batched_acc_PoI(Obs_2_PoI_NN_preds, PoI)
    
    PoI_2_Obs_NN_preds = PoI_2_Obs_NN(init_PoI_2_Obs_NN_params_trained, PoI)
    PoI_2_Obs_NN_Err_all = batched_acc_obs(PoI_2_Obs_NN_preds, Observations)
    
    return Obs_2_PoI_NN_Err_all.mean(), PoI_2_Obs_NN_Err_all.mean()


## Generate the table

In [7]:
import numpy
print(numpy.__version__)

1.21.6


In [8]:

name_learning_approach_array = ['$\purePOP$', '$\pureOPO$', '$\mcPOP$', '$\mcOPO$', '$\mcOPOfull$', '$\TNetAE$', '$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [1, 100]:
        if name_learning_approach == '$\purePOP$':
            name_learning = 'Naive_AutoEncoder_P2O2P'
            best_inv = 'STEP_2'
            best_for = 'STEP_1'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            
        if (name_learning_approach == '$\mcPOP$'):
            name_learning = 'mc_AutoEncoder_P2O2P'
            best_inv = 'STEP_2'
            best_for = 'STEP_1'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            
        if name_learning_approach == '$\pureOPO$':
            name_learning = 'Naive_AutoEncoder_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            
        if (name_learning_approach == '$\mcOPO$'):
            name_learning = 'mc_AutoEncoder_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            
        if (name_learning_approach == '$\TNetAE$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            
        if (name_learning_approach == '$\TNetAEfull$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            
        if (name_learning_approach == '$\mcOPOfull$'):
            name_learning = 'mc_AutoEncoder_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            
            
        filename_pattern = f'Foward_{best_for}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')
        
        # print(matching_files[0])
        
        best_for_params = pickle.load(open(matching_files[0], 'rb'))
        opt_int, opt_update, opt_get_params = optimizers.adam(1e-3)
        # For_opt_state = optimizers.pack_optimizer_state(best_for_params)
        if best_for == 'STEP_1':
            _, init_PoI_2_Obs_NN_params_trained  = best_for_params
        if best_for == 'STEP_2':
            init_PoI_2_Obs_NN_params_trained  = best_for_params
        
        filename_pattern = f'Inverse_{best_inv}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')
        best_inv_params = pickle.load(open(matching_files[0], 'rb'))
        # opt_int, opt_update, opt_get_params = optimizers.adam(1e-3)
        # For_opt_state = optimizers.pack_optimizer_state(best_inv_params)
        if best_inv == 'STEP_1':
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
        if best_inv == 'STEP_2':
            init_Obs_2_PoI_NN_params_trained  = best_inv_params
        
        Obs_2_PoI_NN_accuracy_test, PoI_2_Obs_NN_accuracy_test = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters)
        print(f'{name_learning_approach} {num_train} {Obs_2_PoI_NN_accuracy_test : .2f} and {PoI_2_Obs_NN_accuracy_test :.2e}')
        
        

$\purePOP$ 1  156.99 and 2.99e-01
$\purePOP$ 100  72.22 and 6.72e-02
$\pureOPO$ 1  103.94 and 5.60e+00
$\pureOPO$ 100  40.20 and 5.94e-01
$\mcPOP$ 1  161.48 and 2.99e-01
$\mcPOP$ 100  76.33 and 6.72e-02
$\mcOPO$ 1  46.43 and 5.15e-01
$\mcOPO$ 100  27.29 and 2.20e-03
$\mcOPOfull$ 1  46.43 and 3.79e-01
$\mcOPOfull$ 100  27.29 and 2.12e-03
$\TNetAE$ 1  25.68 and 2.14e-03
$\TNetAE$ 100  24.54 and 1.49e-03
$\TNetAEfull$ 1  25.68 and 2.10e-03
$\TNetAEfull$ 100  24.54 and 1.45e-03


## Plot quantative comparison plots for inverse and forward observations for different approaches

In [9]:
def plot_single_true_versus_test(True_solutions, Pred_solutions, filename = ' ', savefig = False, lim_min = 100., lim_max = 100.):
    fig = plt.figure()
    fig.patch.set_facecolor('xkcd:white')
    
    if lim_min == 100.:
        lim_min = jnp.round(jnp.asarray([True_solutions.min(), Pred_solutions.min()]).min(),1) -.2
        lim_max = jnp.round(jnp.asarray([True_solutions.max(), Pred_solutions.max()]).max(),1) +.2
    
    plt.scatter(True_solutions, Pred_solutions)
    plt.gca().set_aspect('equal', adjustable='box')
    
    lim_min = -3
    lim_max = 3
    
    axis_range = np.linspace(lim_min, lim_max, 5)
    plt.xticks(axis_range), plt.yticks(axis_range)
    
    plt.xlim([-3.,3.])
    plt.ylim([-3.,3.])
    
    x = np.linspace(lim_min, lim_max, 1000)
    plt.plot(x,x, 'r', linewidth=3)
    
    current_values = plt.gca().get_xticks()
    # plt.gca().set_yticklabels(['{:,.1f}'.format(x) for x in current_values])
    # plt.gca().set_xticklabels(['{:,.1f}'.format(x) for x in current_values])
    
    plt.xticks([-3,0,3]), plt.yticks([-3,0,3])
    plt.xticks([]), plt.yticks([])
    if savefig:
        # tikzplotlib.save(f"{filename}.tex")
        plt.savefig(f'{filename}.pdf', bbox_inches='tight')
        plt.close()
    else:
        plt.show()



N = 32
x = np.linspace(0, 1, N)
y = np.linspace(0, 1, N)
X, Y = np.meshgrid(x, y)
XX = X.flatten()
YY = Y.flatten()

def plot_domain(nodal_values, top, bottom, de, filename = '', save = False, cmap = cm.Reds):
    L = np.sort(np.asarray((list(set(np.round(np.linspace(bottom, top, 30),3))))))
    fig = plt.figure()
    fig.patch.set_facecolor('xkcd:white')
    ax = fig.add_subplot(1,1,1)
    surf = ax.contourf(X, Y, (np.reshape(nodal_values, (32,32))), vmin = bottom, vmax = top, cmap=cmap, levels = L)
    cbar = fig.colorbar(surf, shrink = 1., orientation = 'vertical', pad = 0.02)
    cbar.set_ticks(np.linspace(bottom, top, de))
    
    for point in obs_locations:
        plt.plot(XX[point,], YY[point,], 'ks', mew=3, ms=1)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xticks([])
    ax.set_yticks([])
    
    if save:
        plt.savefig(filename, bbox_inches='tight')
        plt.close()
    else:
        plt.show()



In [13]:

name_learning_approach_array = ['$\purePOP$', '$\pureOPO$', '$\mcPOP$', '$\mcOPO$', '$\mcOPOfull$', '$\TNetAE$', '$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [1, 100]:
        if name_learning_approach == '$\purePOP$':
            name_learning = 'Naive_AutoEncoder_P2O2P'
            best_inv = 'STEP_2'
            best_for = 'STEP_1'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            max_mean = 0.6
            max_std = 0.3
            
        if (name_learning_approach == '$\mcPOP$'):
            name_learning = 'mc_AutoEncoder_P2O2P'
            best_inv = 'STEP_2'
            best_for = 'STEP_1'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            max_mean = 0.6
            max_std = 0.3
            
        if name_learning_approach == '$\pureOPO$':
            name_learning = 'Naive_AutoEncoder_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            max_mean = 0.6
            max_std = 0.3
            
        if (name_learning_approach == '$\mcOPO$'):
            name_learning = 'mc_AutoEncoder_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            max_mean = 0.6
            max_std = 0.3
            
        if (name_learning_approach == '$\TNetAE$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            max_mean = 0.6
            max_std = 0.3
            
        if (name_learning_approach == '$\TNetAEfull$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            max_mean = 0.6
            max_std = 0.3
        if (name_learning_approach == '$\mcOPOfull$'):
            name_learning = 'mc_AutoEncoder_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O
            max_mean = 0.6
            max_std = 0.3
            
        filename_pattern = f'Foward_{best_for}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')
        
        best_for_params = pickle.load(open(matching_files[0], 'rb'))
        opt_int, opt_update, opt_get_params = optimizers.adam(1e-3)
        if best_for == 'STEP_1':
            _, init_PoI_2_Obs_NN_params_trained  = best_for_params
        if best_for == 'STEP_2':
            init_PoI_2_Obs_NN_params_trained  = best_for_params
        
        filename_pattern = f'Inverse_{best_inv}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')
        best_inv_params = pickle.load(open(matching_files[0], 'rb'))
        if best_inv == 'STEP_1':
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
        if best_inv == 'STEP_2':
            init_Obs_2_PoI_NN_params_trained  = best_inv_params
        
        # Obs_2_PoI_NN_accuracy_test, PoI_2_Obs_NN_accuracy_test = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters)
        # print(f'{name_learning_approach} {num_train} {Obs_2_PoI_NN_accuracy_test : .2f} and {PoI_2_Obs_NN_accuracy_test :.2e}')
        
        PoI_test_pred = PoI_2_Obs_NN(init_Obs_2_PoI_NN_params_trained, test_Observations)
        Obs_test_pred = Obs_2_PoI_NN(init_PoI_2_Obs_NN_params_trained, test_Parameters)
        
        file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/forward_predictions_{name_learning}_d{num_train}'
        if (name_learning_approach == '$\TNetAEfull$') or (name_learning_approach == '$\mcOPOfull$'):
            plot_single_true_versus_test(batched_Observation_operator(Obs_test_pred), test_Observations, filename = file_name_to_save_fig, savefig = True)
        else:
            plot_single_true_versus_test(Obs_test_pred, test_Observations, filename = file_name_to_save_fig, savefig = True)
        
        true_PoI = np.einsum('bi, ji -> bj', test_Parameters, Basis)
        pred_PoI = np.einsum('bi, ji -> bj', PoI_test_pred, Basis)
        nodal_values = np.abs(true_PoI - pred_PoI).mean(axis = 0)
        nodal_values_std = np.abs(true_PoI - pred_PoI).std(axis = 0)
        
        file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/inverse_mean_error_predictions_{name_learning}_d{num_train}.pdf'
        plot_domain(nodal_values, max_mean, 0., 4, filename = file_name_to_save_fig, save = True)
        
        file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/inverse_std_error_predictions_{name_learning}_d{num_train}.pdf'
        plot_domain(nodal_values_std, max_std, 0., 4, filename = file_name_to_save_fig, save = True)
        
        

In [14]:
true_PoI = np.einsum('bi, ji -> bj', test_Parameters, Basis)
pred_PoI = np.einsum('bi, ji -> bj', Tik_POI, Basis)
nodal_values = np.abs(true_PoI - pred_PoI).mean(axis = 0)
nodal_values_std = np.abs(true_PoI - pred_PoI).std(axis = 0)

file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/inverse_mean_error_predictions_Tikhonov.pdf'
plot_domain(nodal_values, max_mean, 0., 4, filename = file_name_to_save_fig, save = True)

file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/inverse_std_error_predictions_Tikhonov.pdf'
plot_domain(nodal_values_std, max_std, 0., 4, filename = file_name_to_save_fig, save = True)



### Plot for naive OPO for proposal

In [10]:

name_learning_approach_array = ['$\purePOP$', '$\pureOPO$', '$\mcPOP$', '$\mcOPO$', '$\mcOPOfull$', '$\TNetAE$', '$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [1]:
            
        if name_learning_approach == '$\pureOPO$':
            name_learning = 'Naive_AutoEncoder_O2P2O'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_sparseO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_sparse_O
            max_mean = 0.6
            max_std = 0.3
            
            
        filename_pattern = f'Foward_{best_for}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')
        
        best_for_params = pickle.load(open(matching_files[0], 'rb'))
        opt_int, opt_update, opt_get_params = optimizers.adam(1e-3)
        if best_for == 'STEP_1':
            _, init_PoI_2_Obs_NN_params_trained  = best_for_params
        if best_for == 'STEP_2':
            init_PoI_2_Obs_NN_params_trained  = best_for_params
        
        filename_pattern = f'Inverse_{best_inv}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')
        best_inv_params = pickle.load(open(matching_files[0], 'rb'))
        if best_inv == 'STEP_1':
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
        if best_inv == 'STEP_2':
            init_Obs_2_PoI_NN_params_trained  = best_inv_params
        
        # Obs_2_PoI_NN_accuracy_test, PoI_2_Obs_NN_accuracy_test = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters)
        # print(f'{name_learning_approach} {num_train} {Obs_2_PoI_NN_accuracy_test : .2f} and {PoI_2_Obs_NN_accuracy_test :.2e}')
        
        PoI_test_pred = PoI_2_Obs_NN(init_Obs_2_PoI_NN_params_trained, test_Observations)
        Obs_test_pred = Obs_2_PoI_NN(init_PoI_2_Obs_NN_params_trained, test_Parameters)
        

In [15]:
PoI_test_pred.shape

    
nodal_values = pred_PoI[sample,:]
max, min, de = .6, -.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/Tnet_full_test_{num_train}sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = False, cmap = 'jet', obs_plot = False)


NameError: name 'pred_PoI' is not defined

## plot the 1st training sample (converted triangulaization plot from Fenics)



In [12]:
def plot_domain_train(nodal_values, top, bottom, de, filename = '', save = False, cmap = 'Reds', obs_plot = True):
    L = np.sort(np.asarray((list(set(np.round(np.linspace(bottom, top, 30),3))))))
    fig = plt.figure()
    fig.patch.set_facecolor('xkcd:white')
    ax = fig.add_subplot(1,1,1)
    surf = ax.contourf(X, Y, (np.reshape(nodal_values, (32,32))), vmin = bottom, vmax = top, cmap=cmap, levels = L)
    cbar = fig.colorbar(surf, shrink = 1., orientation = 'vertical', pad = 0.02)
    cbar.set_ticks(np.linspace(bottom, top, de))
    
    if obs_plot:
        for point in obs_locations:
            plt.plot(XX[point,], YY[point,], 'ks', mew=3, ms=1)
        
    ax.set_aspect('equal', adjustable='box')
    ax.set_xticks([])
    ax.set_yticks([])
    
    if save:
        plt.savefig(filename, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

In [13]:
nodal_values = train_full_Observations_synthetic[0,:]

max, min, de = 1.6, -1.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/True_train_sample_yobs_sample.pdf'
plot_domain_train(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap = 'jet')

In [14]:
nodal_values = np.einsum('bi, ji -> bj', train_Parameters, Basis)[0,:]
max, min, de = .6, -.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/True_train_sample_POI_sample.pdf'
plot_domain_train(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap = 'jet', obs_plot = False)

## predictions by TNet full

In [15]:

name_learning_approach_array = ['$\mcOPOfull$', '$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [100, 1]:
        if (name_learning_approach == '$\mcOPOfull$'):
            name = 'mc_full'
            name_learning = 'mc_AutoEncoder_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O


        if (name_learning_approach == '$\TNetAEfull$'):
            name = 'Tnet_full'
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            best_for = 'STEP_2'
            Obs_2_PoI_NN, PoI_2_Obs_NN, _ = neuralnetwork_models_P2_fullO(num_observation, num_truncated_series, neurons, key_network)
            acc_function = accuracy_data_P2_full_O


        filename_pattern = f'Foward_{best_for}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')

        best_for_params = pickle.load(open(matching_files[0], 'rb'))
        opt_int, opt_update, opt_get_params = optimizers.adam(1e-3)
        if best_for == 'STEP_1':
            _, init_PoI_2_Obs_NN_params_trained  = best_for_params
        if best_for == 'STEP_2':
            init_PoI_2_Obs_NN_params_trained  = best_for_params

        filename_pattern = f'Inverse_{best_inv}{name_learning}==data-d{num_train}_to_100_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')
        best_inv_params = pickle.load(open(matching_files[0], 'rb'))
        if best_inv == 'STEP_1':
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
        if best_inv == 'STEP_2':
            init_Obs_2_PoI_NN_params_trained  = best_inv_params

        PoI_test_pred = PoI_2_Obs_NN(init_Obs_2_PoI_NN_params_trained, test_Observations)
        Obs_test_pred = Obs_2_PoI_NN(init_PoI_2_Obs_NN_params_trained, test_Parameters)

        true_PoI = np.einsum('bi, ji -> bj', test_Parameters, Basis)
        pred_PoI = np.einsum('bi, ji -> bj', PoI_test_pred, Basis)
        nodal_values = np.abs(test_Observations_full - Obs_test_pred).mean(axis = 0)
        nodal_values_std = np.abs(test_Observations_full - Obs_test_pred).std(axis = 0)

        mean_max = 0.04
        std_max = 0.04
        colorbar_points = 5
        if (name_learning_approach == '$\mcOPOfull$') and (num_train == 1):
            mean_max = 0.9
            std_max = 1.5
            colorbar_points = 3
            
        file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/{name}_trained_with{num_train}sample_mean_error_test_500sample_yobs_sample.pdf'
        plot_domain(nodal_values, mean_max, 0., colorbar_points, filename = file_name_to_save_fig, save = True)

        file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/{name}_trained_with{num_train}sample_std_error_test_500sample_yobs_sample.pdf'
        plot_domain(nodal_values_std, std_max, 0., colorbar_points, filename = file_name_to_save_fig, save = True)
        
        

## TNet full and plot cherry pick sample

In [13]:
def plot_domain(nodal_values, top, bottom, de, filename = '', save = False, cmap = 'Reds', obs_plot = True):
    L = np.sort(np.asarray((list(set(np.round(np.linspace(bottom, top, 30),3))))))
    fig = plt.figure()
    fig.patch.set_facecolor('xkcd:white')
    ax = fig.add_subplot(1,1,1)
    surf = ax.contourf(X, Y, (np.reshape(nodal_values, (32,32))), vmin = bottom, vmax = top, cmap=cmap, levels = L)
    cbar = fig.colorbar(surf, shrink = .6, orientation = 'horizontal', pad = 0.02)
    cbar.set_ticks(np.linspace(bottom, top, de))
    if obs_plot:
        for point in obs_locations:
            plt.plot(XX[point,], YY[point,], 'ks', mew=3, ms=1)
        
    ax.set_aspect('equal', adjustable='box')
    ax.set_xticks([])
    ax.set_yticks([])
    
    if save:
        plt.savefig(filename, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

In [17]:
AA = np.abs(test_Observations_full - Obs_test_pred).sum(axis = 1)
np.where(AA == AA.min())

BB = np.abs(true_PoI - pred_PoI).sum(axis = 1)
np.where(BB == BB.min())

CC = AA / AA.max() + BB / BB.max()
np.where(CC == CC.min())

(array([310]),)

In [14]:
sample = 310
full_true_obs = test_Observations_full

In [19]:
nodal_values = full_true_obs[sample,:]

max, min, de = 1.6, -1.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/True_test_{num_train}sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap = 'jet')

In [20]:
nodal_values = Obs_test_pred[sample,:]

max, min, de = 1.6, -1.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/Tnet_full_test_{num_train}sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap = 'jet')

In [21]:
nodal_values = np.abs(full_true_obs[sample,:] - Obs_test_pred[sample,:])

max, min, de = 0.020, 0., 3
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/Tnet_full_error_test_{num_train}sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap = 'Reds')

In [28]:

nodal_values = true_PoI[sample,:]
max, min, de = .6, -.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/True_test_sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap = 'jet', obs_plot = False)



In [24]:
    
nodal_values = pred_PoI[sample,:]
max, min, de = .6, -.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/Tnet_full_test_{num_train}sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap = 'jet', obs_plot = False)


In [25]:

nodal_values = np.abs(true_PoI[sample,:] - pred_PoI[sample,:])
max, min, de = .3, 0., 4
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/Tnet_full_test_error_{num_train}sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, obs_plot = False)


In [26]:
TIK_PoI = np.einsum('bi, ji -> bj', Tik_POI, Basis)
nodal_values = TIK_PoI[sample,:]
max, min, de = .6, -.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/Tik_test_sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, cmap = 'jet', obs_plot = False)


In [27]:
    
nodal_values = np.abs(true_PoI[sample,:] - TIK_PoI[sample,:])
max, min, de = .3, 0., 4
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/Tik_test_error_sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = True, obs_plot = False)


## test

In [ ]:
nodal_values = pred_PoI[37,:]
max, min, de = .6, -.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/Tnet_full_test_{num_train}sample_POI_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = False, cmap = 'jet')


## plot training samples

In [ ]:
train_Observations_full = load_data('data/Test_full_Observations' + str(num_test) + '.csv', (num_test, -1))


In [ ]:
nodal_values = train_Observations_full[0,:]
nodal_values_mean = train_Observations_full.mean(axis = 0)
nodal_values = train_Observations_full.mean(axis = 0)

max, min, de = 1.6, -1.6, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/True_test_sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = False, cmap = 'jet')

In [ ]:
nodal_values = train_Observations_full[91,:]
nodal_values = train_Observations_full[91,:] - nodal_values_mean


max, min, de = .5, -.5, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/True_test_sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = False, cmap = 'jet')

In [ ]:
nodal_values = train_Observations_full[0,:]
nodal_values = train_Observations_full[0,:] - nodal_values_mean

max, min, de = .5, -.5, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/True_test_sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = False, cmap = 'jet')

In [ ]:
num_train_position = 90
num_train = 1
nodal_values = train_Observations_full[num_train_position:num_train_position+num_train,:]
nodal_values = nodal_values - nodal_values_mean

max, min, de = .5, -.5, 5
file_name_to_save_fig = f'../../mcAE_Overleaf/Figs/2D_NS/True_test_sample_yobs_sample.pdf'
plot_domain(nodal_values, max, min, de, filename = file_name_to_save_fig, save = False, cmap = 'jet')

## Randomness over 10 different samples for training 1 sample.

In [ ]:
Error_array = []
name_learning_approach_array = ['$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [1]:
        if (name_learning_approach == '$\TNetAEfull$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            
        filename_pattern = f'Inverse_Random_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')
            
        for matching_file in matching_files:
            best_inv_params = pickle.load(open(matching_file, 'rb'))
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
            Obs_2_PoI_NN_accuracy_test, PoI_2_Obs_NN_accuracy_test = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters)
            Error_array.append(Obs_2_PoI_NN_accuracy_test)
            

In [ ]:
jnp.asarray(Error_array).mean(), jnp.asarray(Error_array).std()

In [ ]:
# Plot the triangulation
plt.figure(figsize=(4, 4))
# plt.triplot(triangulation, 'k-', linewidth=0.5)
plt.plot([0,0], [0,1], 'k:', linewidth=1.)
plt.plot([0,1], [0,0], 'k:', linewidth=1.)
plt.plot([0,1], [1,1], 'k:', linewidth=1.)
plt.plot([1,1], [0,1], 'k:', linewidth=1.)
plt.plot(XX[obs_locations,], YY[obs_locations,], 'ks', markersize=3, label = 'Observed points')

# Add index next to each point in obs_locations
for i, loc in enumerate(obs_locations):
    plt.text(XX[loc] + 0.01, YY[loc] - 0.01, str(i+1), fontsize=8)

plt.axis('equal')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])

plt.xticks([])
plt.yticks([])

# Hide the axis lines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_visible(False)

import tikzplotlib
tikzplotlib.save(f"../../mcAE_Overleaf/Figs/2D_NS/domain_observables_index.tex")

# plt.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

list_point = np.asarray([0, 1, 8, 17, 28, 29, 30, 31, 41, 40, 50, 51])

plt.figure(figsize=(10, 4))  # Increased figure size for better visibility

# Plot individual scatter points
for point in list_point:
    plt.scatter(np.arange(1, len(train_Observations.mean(axis=0))+1), train_Observations[point], c='r', alpha=0.5, s=20)

plt.scatter(np.arange(1, len(train_Observations.mean(axis=0))+1), train_Observations[point], c='r', alpha=0.5, s=20, label = 'samples')

# Calculate mean and standard deviation
x = np.arange(1, len(train_Observations.mean(axis=0))+1)  # Start x from 1
y = train_Observations.mean(axis=0)
yerr = train_Observations.std(axis=0)

# Plot errorbar
plt.errorbar(x, y, yerr=yerr, fmt='o', color='b', ecolor='lightblue', capsize=5, label='Mean with Std Dev')

plt.xlabel('Observation Index')
plt.ylabel('Observation Magnitude')
plt.xticks(np.arange(1, 22, 4))  # Set x-ticks starting from 1
plt.yticks([-1.5, 0, 1.5])
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
# plt.show()

import tikzplotlib
tikzplotlib.save(f"../../mcAE_Overleaf/Figs/2D_NS/observations_mean_standard_deviation.tex")

## Different noise level for 1st training sample for TNet full

In [ ]:
Error_array = []
name_learning_approach_array = ['$\TNetAEfull$']
for name_learning_approach in name_learning_approach_array:
    for num_train in [1]:
        if (name_learning_approach == '$\TNetAEfull$'):
            name_learning = 'mc_AutoEncoder_TNet_O2P2O_modified'
            best_inv = 'STEP_1'
            
        filename_pattern = f'Inverse_noise_*'
        matching_files = glob.glob(f'Network_picked/Network_picked/{filename_pattern}')
        noise_levels = np.sort([float(file.split('noise_')[1].split('_')[0]) for file in matching_files])
        
        # for matching_file in matching_files:
        #     print(matching_file)
        for noise_level in noise_levels:
            matching_file = f'Network_picked/Network_picked/Inverse_noise_{noise_level}_STEP_1mc_AutoEncoder_TNet_O2P2O==data-d1_to_100_alpha_mcDNN_500_alpha_P2O_1.0_alpha_O2P_1.0_alpha_Res_0.0noise_level_extra_{noise_level}'
            print(noise_level)
            best_inv_params = pickle.load(open(matching_file, 'rb'))
            init_Obs_2_PoI_NN_params_trained, _  = best_inv_params
            Obs_2_PoI_NN_accuracy_test, PoI_2_Obs_NN_accuracy_test = acc_function((init_Obs_2_PoI_NN_params_trained, init_PoI_2_Obs_NN_params_trained), test_Observations, test_Parameters)
            Error_array.append(Obs_2_PoI_NN_accuracy_test)
            
Error_array
            

In [ ]:
plt.figure(figsize=(3, 2))  
plt.plot(noise_levels, Error_array, 'b')
plt.scatter(noise_levels, Error_array, marker='*', color='r')
plt.ylim(20, 80), plt.xlim(-0.02, 0.72)
plt.ylabel('Relative error'), plt.xlabel('Noise level')
plt.yticks([20, 40, 60, 80])
plt.xticks([0., 0.2, 0.4, 0.6, 0.7])
# plt.show()

import tikzplotlib
tikzplotlib.save(f"../../mcAE_Overleaf/Figs/2D_NS/TNet_full_noise_level_versus_relative_error.tex")